# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_info_df = pd.read_csv('output_data/cities.csv')
weather_info_df.head()
weather_info_df = weather_info_df.rename(columns={"City Name" : "City"})
weather_info_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,goderich,43.75,-81.72,47.98,87,100,23.67,CA,1603504213
1,1,jamestown,42.10,-79.24,67.44,60,75,12.75,US,1603504426
2,2,souillac,-20.52,57.52,73.40,73,40,11.41,MU,1603504369
3,3,saint-philippe,-21.36,55.77,68.56,73,20,5.82,RE,1603504374
4,4,cabra,37.47,-4.44,57.99,81,0,0.29,ES,1603504384


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
# fig = gmaps.figure()
# fig

In [17]:
locations = weather_info_df[["Lat", "Lng"]]
humidity = weather_info_df["Humidity"].astype(float)
fig = gmaps.figure(center = [0,0] ,zoom_level = 2)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                               dissipating=False, max_intensity=75,
                               point_radius = 3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
 Narrow down the DataFrame to find your ideal weather condition. For example:
  * A max temperature lower than 90 degrees but higher than 75.
  * Wind speed less than 10 mph.
  * 10% cloudiness.
  * Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
    

In [11]:
weather_info_df['Cloudiness'].value_counts()
clouds_df = weather_info_df.where(weather_info_df['Cloudiness']<=10, other=np.nan)
clouds_df.dropna()
#143 possibilities
wind_df = clouds_df.where(clouds_df['Wind Speed']<=10)
wind_df.dropna()
#115 possibilities
temp_df = wind_df.where(wind_df['Max Temp']>=75)
temp_df.dropna()
# low_temp_df = temp_df.where()
clouds2_df = temp_df.where(temp_df['Cloudiness']>0)
clouds2_df.dropna()
ideal_df = clouds2_df.dropna()
ideal_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
68,68.0,barabai,-2.58,115.38,90.25,53.0,2.0,2.55,ID,1.603505e+09
122,122.0,cabo san lucas,22.89,-109.91,82.89,66.0,5.0,3.36,MX,1.603505e+09
203,203.0,sorong,-0.88,131.25,83.70,77.0,3.0,6.96,ID,1.603505e+09
256,256.0,kavali,14.92,79.98,81.03,72.0,1.0,6.55,IN,1.603505e+09
291,291.0,luwuk,-0.95,122.79,87.42,60.0,6.0,3.74,ID,1.603505e+09
300,300.0,port blair,11.67,92.75,83.17,76.0,7.0,6.06,IN,1.603505e+09
350,350.0,wewak,-3.55,143.63,85.12,64.0,4.0,3.76,PG,1.603505e+09
393,393.0,oriximina,-1.77,-55.87,82.08,61.0,8.0,2.62,BR,1.603505e+09
450,450.0,bitkine,11.98,18.21,79.41,23.0,1.0,4.94,TD,1.603505e+09
467,467.0,presidencia roque saenz pena,-26.79,-60.44,77.00,75.0,2.0,3.00,AR,1.603505e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [21]:
hotel_df = ideal_df.reset_index(drop=True)
hotel_df["Hotel Name"] = ""

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    hotel_name = requests.get(base_url, params=params)
    hotel_name = hotel_name.json()
    print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df

{
    "html_attributions": [],
    "next_page_token": "AcnaJv197s6U5yT96FO0wZ9xk9eLV3MU7BDmhfOOucQqd27Pw3CQz8huMFbwlWEZE4ise6MBHOCas-RtU83cPI2H6rT5hdnD5F1rqdpsC54pOBddol0d1l-QWKGOALx8_hKIyZ_2cFPoesX8Y54OAHzfMLqCcaqobFVnQxwaendW3QNfhJGVyZa771d5dpwi9EVMFXsvLNXClghtRNmo4gK-j2v7aLfiB4Wfmd0fbmKmaCRRYn9in7RD1q5mROBgF8Y3CVgnnuW4umetPV8HMzytrnizHMF0oszXbH8SJt7ZVcdeXAaPEcRK0OEAjmNGgSt1mKzq0cjv0Ugg469SuiBZVvh7QcOdXIBcos3ch1vE9632NwCDVBSNy0aEkfAe76na7DiJa77K1GgUbfPZLZB97av_RSa7MrvOvZQdXx7VFtyTQsk_",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -2.587194,
                    "lng": 115.389286
                },
                "viewport": {
                    "northeast": {
                        "lat": -2.585946269708498,
                        "lng": 115.3906660802915
                    },
                    "southwest": {
                        "lat": -2.588644230291502,
        

{
    "html_attributions": [],
    "next_page_token": "AcnaJv1aaDrgSppO_7x_NFSCypB9YqlqYch0ScFoALaR-UacoJsHiuLOCMbvmAOCbZVgL9bB5fi9UGPZRmTtDsGPX26UUv18inl2h8fKtq1ibVfriCGXHk_VO4CYl5ATnfAibV11HCRMwvDF5cw02uLPNnc8gPbgVodcXaLolbte-mttOdzz7oYyh6xdwv1XZlEaTeyviT8IaMgb6HUtPR6263_i0cwCFdHcI9A0_darP1WjWXC0FAXYJj6KLOwQgpy7pDb4f2xzqr4--XsvaMFfCh3uP6LP7ABuZ9bM5ZPmuHt1-esgXqNyP0Wh4_GCReCeYfHcKUT0iS0rhxMIZLUSkEqecGhDRG0fZYs7wTydkgPk6QPOTLZ7HRn9MueplGug-T2sACqtlA8LxDG3ox_yunZgKfv4JIByNX5IaMdQHkYdTm97",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 22.8905327,
                    "lng": -109.9167371
                },
                "viewport": {
                    "northeast": {
                        "lat": 22.9202876,
                        "lng": -109.8942559
                    },
                    "southwest": {
                        "lat": 22.872165,
                        "lng": -109.9717403
                    }
       

{
    "html_attributions": [],
    "next_page_token": "AcnaJv3JYym7GmKLdbo8QK67c-YdT0MZjRJHLcqMXH8LZlxNkR978ns_OvCVD1toIZ5tVEJfGUqmbuHLbnzPo4RJ5SKI_ZS4_YUGSX4YcWoAcRykKKD6RAr5V9DzJH5tSgOsD44iamTk1vxOOezTMNhrAAbBrf5m_KenbWPe4WPtpZ3fiEoG_9xGQEg2OXWeRArWVOpioISBC5HB2cZMWYqN2zZAFUnYzUDbIMheittOXEbDSWtDDbnX1fMsYbffW0BaTGIXfTaPW9c7o7g5KNCbXqevOpNhlsq0aEDTRnkNVQr8Yny6jwtUQxDxzAHPM5me4UMKPhesl4-nxozy7XZUXWVPnkx7I2bfouhrJcQh6zVzpgEuDgOqA60CnvvSEftMHQVPxPxL6MPujUFCtDDIFIc8Vs_PGbLgYSrJ2-XyM6rxykqS",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -0.8761629000000001,
                    "lng": 131.255828
                },
                "viewport": {
                    "northeast": {
                        "lat": -0.8340122,
                        "lng": 131.3439695
                    },
                    "southwest": {
                        "lat": -0.951802,
                        "lng": 131.184822
                    }
   

{
    "html_attributions": [],
    "next_page_token": "AcnaJv24A-iq2yN-z_7WQ8FjNG6t4_wLOcPzno1Jb9W7FZOUXoV2EVgGJbCCP5KUmBF652RGgotdz7IPYcVWbjr3kEifE1CR0M_CYTg83s4kEQ8t-GjONXJVSAeMr3mPhbkTKS-9T-FZ2RWhA5gkCEbsCpFIMWODqODUuYY83-tFmY1I99t40yl6k70E-mkQksGLwmI96grIpKZTQEE26g8nrPILD-tQvlfKLhJO1qJUwbMw9ALvK3ucGme2LfznKZM1Mx7OhhzK2gJ9QLn6xaRYJBwina1I54CfW6yw5tze7fjMpL9-iRipY6cnL79SvHodqQDD0hVISGBJ_jgpf6Zgq2u9cHjPMcfP51JO2hYgEVKFAGOgkmp3dxd89lX2394R46xGIuhqXmhd2aBTDPN3-tFtW2SCi7lsZpqhosdbtdAWkVk5",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 14.9131806,
                    "lng": 79.9929798
                },
                "viewport": {
                    "northeast": {
                        "lat": 14.9427117,
                        "lng": 80.0181484
                    },
                    "southwest": {
                        "lat": 14.8849859,
                        "lng": 79.96665
                    }
              

{
    "html_attributions": [],
    "next_page_token": "AcnaJv1-l-I8gsStmdAyOkGch3aGje4ePhCPHYwBplB9SBS2h1fu9R1jsABKA3SvVtEJ1Yp-Blt3D9zfcXMsj_UX5qv_-0nO4AEyypxqIHDl9S4VBUf-ZmqZ3eLNtW_7MNKIBYEb3ZGrs1obycBLmb1Au5paj9x-_oQiimhet4bKwGTftVVTMWK4QmpU4IJ9DI_ulEDnj2jjncCfHXkytxY7XrvV05wD68QmVv9t1PNIKOQDbt7xQzJ3PfuHuvEGyO1f3QHKWNIaau1-vzMPW-sFUZHPiT28sAtUKGtbSr0qNauifX6PFZ8YLuuJRr-cp83cNfS3CxVAnuQKUu0QM6Ss3KhVDiBCe58UJgCy0yHocmR4_CwmPYSE1w028eQCStHN8vmU_2NAD2oSmnSvrEESk09xquCjB6ZCIahd6djeVmHai-vq",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -0.938828,
                    "lng": 122.792801
                },
                "viewport": {
                    "northeast": {
                        "lat": -0.9276715,
                        "lng": 122.8088084
                    },
                    "southwest": {
                        "lat": -0.9499845000000001,
                        "lng": 122.7767936
                    }
  

...

{
    "html_attributions": [],
    "next_page_token": "AcnaJv2iPThBc488PbhXs6cOfI0J_cQ8HncwLqr_i4AakrJA095g9ctK5z9iECvhQmQ-2xEw7enGxmsmiywJY979BYwiVQQZ28ad9pJzz6pXYVJZp95HM7DIgDLnp6wigsLJkhudW7vGpieWMEZzVKGS_DjWFbvxVtsPV0GPzgUwvrI3yrsNo9eAdFMGwZcpGc032OCEzkdNKh-k95-l020wDUpbQtPVSoBsTLcK_3fr3LwGldZoBFdMtz7lhr0WMl9EJAtiVeTkQqiTrpWRjOoxFKaffUmJF2opETGCgPnvbfSBkTcgzsNDXkY1OOMdF-IcZYvGokbdenNcE8XLHGTS6QTIjYEXlxWjkZJDtbaNG84z1znfW2jig_x5ecKfeA2_btsP_h1N8GQXTRLg1ic1-PqeA1usQHPy0XzBzHPCqM20Kreg",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -3.5800229,
                    "lng": 143.6583166
                },
                "viewport": {
                    "northeast": {
                        "lat": -3.5471742,
                        "lng": 143.7050344
                    },
                    "southwest": {
                        "lat": -3.5966879,
                        "lng": 143.6037541
                    }
         

{
    "html_attributions": [],
    "next_page_token": "AcnaJv12rTUlKc0SpUbVbl9b2-fgvWPrRcge0eD_2ADg1-mVWvkUZVdm9QI9twzAfFi8G29PmbgbBWGF_qXJoG04UckYaxwiA8XLpZPl2sO9OaOBf_l-LF_Jw98lz6q6fY9VO0tUaiuPRJB4PsAuaN2Vqu-mtrf7dd3ODHb6rq7LV3JePuQ31s5xSr6jRX53xOLuNm6GP_vTAkmXvCduWbDopqQjP7L5-2FLE4MuGq01r1mdT16p2yMPaEusX7KVB0GPm8i_9dpb3wxnW5EG51rpcy1PgD-xYmhcgHJ1IF6cZKD7URI7ZJ-N1fPtiJ7116kpanBdxQduMzGHUqyd02-KkxGeyUqpS4nEB7haiCZ0dHhDeOuVRiQZHHEsgT9p0kZbtgHtznLiTpCoKSQwcrWpXPE2NloMtz1M-NfiMvOk0YF6x2XV",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -1.7618311,
                    "lng": -55.86381549999999
                },
                "viewport": {
                    "northeast": {
                        "lat": -1.7395899,
                        "lng": -55.8473458
                    },
                    "southwest": {
                        "lat": -1.7871818,
                        "lng": -55.87943869999999
                 

{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 11.9781075,
                    "lng": 18.2110626
                },
                "viewport": {
                    "northeast": {
                        "lat": 11.9896218,
                        "lng": 18.2191086
                    },
                    "southwest": {
                        "lat": 11.9729135,
                        "lng": 18.205719
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/geocode-71.png",
            "name": "Bitkine",
            "photos": [
                {
                    "height": 480,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/101474433452185611591\">Mahamat Abba Daninki</a>"
                    ],
                    "photo_reference": "CmRaAAAAU

{
    "html_attributions": [],
    "next_page_token": "AcnaJv374DEoXVGPI1m93tkebXwCI-Gy7QCZ8RMu1Y-dP0CMPMrVt7D9cOUlw2j6B80JzUbVaBLfHxwxXPi1cL3OH2VQK5GQ_0gj9fL6435gwhWVUhvkNR0lXybRvx7vQT9IC52z6CaEExXLJ9yIQ-CnPQ-gkuXL747NrGbUjcHkGW9Ra91ms8yRLozNNQSqY8za6h5IiVI9V2dKz3uEkV1sU7H6CSdJC2KtH_6_QW1kvEHVdr_o87ghNlcEHVICp-jc_Tu3iUK-ZySTYIBCrolUxmWhNbyOgYvCTxs9MYtpxJbTt5u8IvW0Uv0yGI3qfPwIAurFpZaBOHQBDTRV9b44IYcvHX47LrjI2pIYqMsMxBH3iNi8DaAXqxZcPGYudOHrs72Zu6-qCHmAOLg34KOU452Jodh9_LqcJi8gvZxUjRZrzsBZ",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -26.80044279999999,
                    "lng": -60.4312354
                },
                "viewport": {
                    "northeast": {
                        "lat": -26.7499592,
                        "lng": -60.3997877
                    },
                    "southwest": {
                        "lat": -26.8319924,
                        "lng": -60.47321100000001
              

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,68.0,barabai,-2.58,115.38,90.25,53.0,2.0,2.55,ID,1.603505e+09,SMP Negeri 1 Hulu Sungai Tengah
1,122.0,cabo san lucas,22.89,-109.91,82.89,66.0,5.0,3.36,MX,1.603505e+09,Cabo San Lucas
2,203.0,sorong,-0.88,131.25,83.70,77.0,3.0,6.96,ID,1.603505e+09,Sorong
3,256.0,kavali,14.92,79.98,81.03,72.0,1.0,6.55,IN,1.603505e+09,Kavali
4,291.0,luwuk,-0.95,122.79,87.42,60.0,6.0,3.74,ID,1.603505e+09,Luwuk
5,300.0,port blair,11.67,92.75,83.17,76.0,7.0,6.06,IN,1.603505e+09,Port Blair
6,350.0,wewak,-3.55,143.63,85.12,64.0,4.0,3.76,PG,1.603505e+09,Wewak
7,393.0,oriximina,-1.77,-55.87,82.08,61.0,8.0,2.62,BR,1.603505e+09,Oriximiná
8,450.0,bitkine,11.98,18.21,79.41,23.0,1.0,4.94,TD,1.603505e+09,Bitkine
9,467.0,presidencia roque saenz pena,-26.79,-60.44,77.00,75.0,2.0,3.00,AR,1.603505e+09,Presidencia Roque Sáenz Peña


In [14]:
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,68.0,barabai,-2.58,115.38,90.25,53.0,2.0,2.55,ID,1.603505e+09,Madani Hotel
1,122.0,cabo san lucas,22.89,-109.91,82.89,66.0,5.0,3.36,MX,1.603505e+09,Waldorf Astoria Los Cabos Pedregal
2,203.0,sorong,-0.88,131.25,83.70,77.0,3.0,6.96,ID,1.603505e+09,Swiss-Belhotel Sorong
3,256.0,kavali,14.92,79.98,81.03,72.0,1.0,6.55,IN,1.603505e+09,Hotel Mourya Paradise
4,291.0,luwuk,-0.95,122.79,87.42,60.0,6.0,3.74,ID,1.603505e+09,Hotel Permai
5,300.0,port blair,11.67,92.75,83.17,76.0,7.0,6.06,IN,1.603505e+09,"Symphony Samudra, Port Blair"
6,350.0,wewak,-3.55,143.63,85.12,64.0,4.0,3.76,PG,1.603505e+09,In Wewak Boutique Hotel
7,393.0,oriximina,-1.77,-55.87,82.08,61.0,8.0,2.62,BR,1.603505e+09,Hotel Mapuera
8,450.0,bitkine,11.98,18.21,79.41,23.0,1.0,4.94,TD,1.603505e+09,
9,467.0,presidencia roque saenz pena,-26.79,-60.44,77.00,75.0,2.0,3.00,AR,1.603505e+09,Apart temporario LHOTAK


In [19]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [20]:
# Add marker layer ontop of heat map

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Display figure
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))